In [1]:
import json
import re
import pandas as pd
import os
import matplotlib.font_manager as font_manager
from mplsoccer import VerticalPitch
import LanusStats as ls

In [ ]:
# Directory path with matches

cartella = '/Users/alfonsomarino/Desktop/partite genoa'
cartella2 = '/Users/alfonsomarino/Desktop/partite napoli'


# Create a list with complete file path
lista_file = [os.path.join(cartella, f) for f in os.listdir(cartella) if os.path.isfile(os.path.join(cartella, f))]

lista_file2 = [os.path.join(cartella2, f) for f in os.listdir(cartella2) if os.path.isfile(os.path.join(cartella2, f))]

In [ ]:
#Import any font you want from your pc
## Create a font properties object with the font file

font_path_regular = '/Users/alfonsomarino/Desktop/Teko/static/Teko-Regular.ttf'
font_normal = font_manager.FontProperties(fname=font_path_regular)

font_path_med = "/Users/alfonsomarino/Desktop/Teko/static/Teko-Medium.ttf"
font_med = font_manager.FontProperties(fname = font_path_med)

font_path_semi = "/Users/alfonsomarino/Desktop/Teko/static/Teko-SemiBold.ttf"
font_semi = font_manager.FontProperties(fname = font_path_semi)

# Scraping HTML file

In [ ]:
def extract_json_from_html(html_path, save_output=False):
    html_file = open(html_path, 'r')
    html = html_file.read()
    html_file.close()
    regex_pattern = r'(?<=require\.config\.params\["args"\].=.)[\s\S]*?;'
    data_txt = re.findall(regex_pattern, html)[0]

    # add quotations for json parser
    data_txt = data_txt.replace('matchId', '"matchId"')
    data_txt = data_txt.replace('matchCentreData', '"matchCentreData"')
    data_txt = data_txt.replace('matchCentreEventTypeJson', '"matchCentreEventTypeJson"')
    data_txt = data_txt.replace('formationIdNameMappings', '"formationIdNameMappings"')
    data_txt = data_txt.replace('};', '}')

    if save_output:
        # save json data to txt
        output_file = open(f"{html_path}.txt", "wt")
        n = output_file.write(data_txt)
        output_file.close()

    return data_txt

In [ ]:
def extract_data_from_dict(data):
    # load data from json
    event_types_json = data["matchCentreEventTypeJson"]
    formation_mappings = data["formationIdNameMappings"]
    events_dict = data["matchCentreData"]["events"]
    teams_dict = {data["matchCentreData"]['home']['teamId']: data["matchCentreData"]['home']['name'],
                  data["matchCentreData"]['away']['teamId']: data["matchCentreData"]['away']['name']}
    players_dict = data["matchCentreData"]["playerIdNameDictionary"]
    # create players dataframe
    players_home_df = pd.DataFrame(data["matchCentreData"]['home']['players'])
    players_home_df["teamId"] = data["matchCentreData"]['home']['teamId']
    players_away_df = pd.DataFrame(data["matchCentreData"]['away']['players'])
    players_away_df["teamId"] = data["matchCentreData"]['away']['teamId']
    players_df = pd.concat([players_home_df, players_away_df])
    players_ids = data["matchCentreData"]["playerIdNameDictionary"]
    return events_dict, players_df, teams_dict

In [ ]:
import json

def process_multiple_files(html_file_paths):
    all_events = []
    all_players = []
    all_teams = {}

    for match_html_path in html_file_paths:
        # Estrai i dati JSON dal file HTML
        json_data_txt = extract_json_from_html(match_html_path)
        data = json.loads(json_data_txt)

        # Estrai i dati dagli eventi, giocatori e squadre
        events_dict, players_df, teams_dict = extract_data_from_dict(data)

        # Aggiungi i dati estratti all'insieme globale
        all_events.extend(events_dict)  # Unisce tutti gli eventi
        all_players.append(players_df)  # Lista dei DataFrame dei giocatori
        all_teams.update(teams_dict)    # Aggiorna il dizionario delle squadre

    # Combina i DataFrame dei giocatori in un unico DataFrame
    combined_players_df = pd.concat(all_players, ignore_index=True)

    return all_events, combined_players_df, all_teams

# Esegui la funzione per elaborare tutti i file
all_events, combined_players_df, all_teams = process_multiple_files(lista_file)
all_events2, combined_players_df2, all_teams2 = process_multiple_files(lista_file2)

In [ ]:
def get_passes_df(events_dict):
    df = pd.DataFrame(events_dict)
    df['eventType'] = df.apply(lambda row: row['type']['displayName'], axis=1)
    df['outcomeType'] = df.apply(lambda row: row['outcomeType']['displayName'], axis=1)

    df_touch = df[["id","minute", "x", "y", "endX", "endY", "teamId", "playerId", "isTouch", "eventType", "outcomeType"]]

    return df_touch

In [ ]:
df_touch = get_passes_df(all_events)
df = df_touch[df_touch["playerId"] == 128839] #WhoScored ID
df

In [ ]:
df_touch2 = get_passes_df(all_events2)
df2 = df_touch2[df_touch2["playerId"] == 336915] # WhoScored ID
df2

In [ ]:
def filter_passes_to_penalty_area(df):

    # Definition penalty area limits
    x_min, x_max = 83.0, 100.0  # penalty_area_right, right
    y_min, y_max = 21.1, 78.9   # penalty_area_bottom, penalty_area_top
    
    # Creating the filter mask
    mask = (
        (df['isTouch'] == True) &
        (df['x'] >= x_min) &
        (df['x'] <= x_max) &
        (df['y'] >= y_min) &
        (df['y'] <= y_max)
    )
    
    return df[mask]


touch_in_box = filter_passes_to_penalty_area(df)
touch_in_box2 = filter_passes_to_penalty_area(df2)

# Library LanusStats for heatmap

In [ ]:
sofascore = ls.SofaScore()

mc = sofascore.get_player_season_heatmap("Serie A", "24/25", 879346) #SofaScore ID
tho = sofascore.get_player_season_heatmap("Serie A", "24/25", 280991) #SofaScore ID

In [ ]:
Mc = mc.loc[mc.index.repeat(mc['count'])].reset_index(drop=True)
Tho = tho.loc[tho.index.repeat(tho['count'])].reset_index(drop=True)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

almond_cmap = LinearSegmentedColormap.from_list("Almond - 100 colors",
                                                  ['blanchedalmond', 'darkred'], N=500)

In [ ]:
pitch = VerticalPitch(pitch_type='opta', corner_arcs=True, pitch_color = "blanchedalmond", line_color="white", line_alpha=1)

fig, axs = pitch.grid(figheight=10, title_height=0.08, endnote_space=0, title_space=0,
                      axis=False, ncols = 2,
                      grid_height=0.82, endnote_height=0.03)
fig.patch.set_facecolor('blanchedalmond')
#ax.set_facecolor('blanchedalmond')

pitch.scatter(touch_in_box["x"], touch_in_box["y"], s= 50, ax = axs["pitch"][0], c= "grey", alpha=.3)
pitch.kdeplot(Tho.x, Tho.y, ax=axs['pitch'][0],
                    # fill using 100 levels so it looks smooth
                    fill=True, levels=1000, zorder = -1,
                    cmap= almond_cmap)

pitch.scatter(touch_in_box2["x"], touch_in_box2["y"], s= 50, ax = axs["pitch"][1], c= "grey", alpha=.3)
pitch.kdeplot(Mc.x, Mc.y, ax=axs['pitch'][1],
                    # fill using 100 levels so it looks smooth
                    fill=True, levels=1000, zorder = -1,
                    cmap= almond_cmap)

#1, 0.5
axs['endnote'].text(1, -0.8, 'Dati Opta via Fbref  |  @AlfoMarino0975', va='center', ha='right', fontsize=15, 
                    color = "grey", fontproperties = font_normal)

axs['endnote'].text(0.225, -0.8, "Ogni punto indica un tocco  |  I valori delle metriche sono p90", color='#003153',
                  va='center', ha='center', fontsize=20, fontproperties = font_normal)

# Aggiungi il titolo per ciascun campo
axs['pitch'][1].text(91, 108, "S. McTominay  | Serie A 24-25", fontsize=32, fontproperties = font_med, color='darkred')
# Aggiungi sottotitolo per il primo campo
axs['pitch'][1].text(0.5, 1.03, "Duelli aerei vinti: 1.45 | Tocchi in area avv.: 2.62 | Pass. progr. ricevuti: 4.18", va='top', ha='center', fontsize=18, 
                     fontproperties = font_normal, color='#003153', transform=axs['pitch'][1].transAxes)

axs['pitch'][0].text(88, 108, "M. Thorsby  |  Serie A 24-25", fontsize=32, fontproperties = font_med, color='darkred')
# Aggiungi sottotitolo per il secondo campo
axs['pitch'][0].text(0.5, 1.03, "Duelli aerei vinti: 5.14 | Tocchi in area avv.: 2.99 | Pass. progr. ricevuti: 2.93", va='top', ha='center', fontsize=18, 
                     fontproperties = font_normal, color='#003153', transform=axs['pitch'][0].transAxes)


# Aggiungi la freccia all'interno del primo campo 'pitch'
axs['pitch'][0].annotate('', xy=(10, 25), xytext=(10, 5),
                         arrowprops=dict(edgecolor='grey', arrowstyle='->', lw=2.5))

axs['pitch'][1].annotate('', xy=(10, 25), xytext=(10, 5),
                         arrowprops=dict(edgecolor='grey', arrowstyle='->', lw=2.5))

axs['title'].text(0.5, 2.3, "Arieti a confronto", color='darkred',
                  va='center', ha='center', fontsize=40, fontproperties = font_semi)

axs['title'].text(0.5, 1.7, "Sono tra i centrocampisti che toccano più palloni in area di rigore avversaria", color='darkorange',
                  va='center', ha='center', fontsize=28, fontproperties = font_med)


# Salva la figura come file immagine
output_path = "/Users/alfonsomarino/Desktop/heatmap_touch.png"
fig.savefig(output_path, dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())